Siemanko
Hejka!!

1.

In [1]:
from sklearn.metrics import (accuracy_score, f1_score)
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.svm import SVC
from sklearn.utils import resample
import tensorflow as tf
from random import sample 
from copy import deepcopy
from random import randint

In [2]:
(x_train_f, y_train_f), (x_test_f, y_test_f) = tf.keras.datasets.fashion_mnist.load_data()

In [3]:
x_train = x_train_f.reshape((x_train_f.shape[0],-1))
x_test = x_test_f.reshape((x_test_f.shape[0],-1))
y_train = y_train_f.reshape((y_train_f.shape[0],))
y_test = y_test_f

scaler = StandardScaler()

scaler.fit(x_train)

x_train_scaled = scaler.transform(x_train)
x_test_scaled = scaler.transform(x_test)

pca = PCA(n_components=50)
pca.fit(x_train_scaled)

x_train_pca = pca.transform(x_train_scaled)
x_test_pca = pca.transform(x_test_scaled)

2.

Pełne dane

In [4]:
def get_full_prediction(x_tr, y_tr, x_te):
    clf = SVC(probability=True)
    clf.fit(x_tr, y_tr)
    y_pred = clf.predict(x_te)
    pred = clf.predict_proba(x_te)
    return (y_pred, pred)

In [ ]:
predictions = get_full_prediction(x_train_pca, y_train, x_test_pca)

In [ ]:
(y_pred, _) = predictions

In [ ]:
accuracy = accuracy_score(y_test, y_pred)
f_score = f1_score(y_test, y_pred, average='weighted')

In [ ]:
print(accuracy)
print(f_score)

Niepełne dane i cechy

In [ ]:
n_samples = [0.1, 0.25, 0.35, 0.5]
n_features = [0.1, 0.2, 0.3]

In [ ]:
def get_minor_predictions(x_tr, y_tr, x_te, samples_perc, features_perc):
    feature_list = [n for n in range(50)]
    predictions = []
    samples_all = x_tr.shape[0]
    features_all = x_tr.shape[1]
    
    for samples in samples_perc:
        for features in features_perc:
            f = sample(feature_list, int(features_all * features))
            x_train_f = x_tr[:,f]
            
            x_train_s, y_train_s = resample(x_tr, y_tr, n_samples=int(samples * samples_all), replace=False, random_state=0)
            
            svm_clf = SVC(probability=True)
            svm_clf.fit(x_train_s, y_train_s)
            
            y_pred = svm_clf.predict(x_te)
            pred = svm_clf.predict_proba(x_te)
            
            predictions.append((y_pred, pred))
            
    return predictions

In [ ]:
pred = get_minor_predictions(x_train_pca, y_train, x_test_pca, n_samples, n_features)

In [ ]:
def average_pred(predictions):
    m = len(predictions[0][0])
    all_results = [[0 for x in range(10)] for y in range(m)] 
    results = [0] * m
    for (_, pred_proba) in predictions:
        for i in range(m):
            for j in range(10):
                all_results[i][j] += pred_proba[i][j]
    for i in range(m):
        results[i] = all_results[i].index(max(all_results[i]))
    return results

In [ ]:
def majority_pred(predictions):
    m = len(predictions[0][0])
    results = [0] * m
    for i in range(m):
        all_results = [0 for x in range(10)] 
        for (pred, _) in predictions:
            all_results[pred[i]] += 1
        results[i] = all_results.index(max(all_results))
        
    return results

In [ ]:
accuracy_score(y_test, majority_pred(pred))

In [ ]:
accuracy_score(y_test, average_pred(pred))

3a.

In [ ]:
clf_7iter = SVC(probability=True,max_iter=7)

In [ ]:
clf_7iter.fit(x_train, y_train)
y_pred_7ter = clf_7iter.predict(x_test)
pred_7iter = clf_7iter.predict_proba(x_test)

In [ ]:
pred_7iter

In [ ]:
from sklearn.metrics import accuracy_score
accuracy_7iter = accuracy_score(y_test, y_pred_7ter)
accuracy = accuracy_score(y_test, y_pred)

In [ ]:
accuracy_7iter

In [ ]:
accuracy

4. 

In [ ]:
def make_noice_data(data, percent):
    

In [27]:
def make_noice_label(labels, percent):
    labels_with_noice = deepcopy(labels)
    arr_size = labels.shape[0]
    indexes = [n for n in range(arr_size)]
    indexes_to_change = sample(indexes, int(arr_size * percent))
    
    for i in indexes_to_change:
        old_val = labels[i]
        new_val = randint(0, 10)
        while old_val == new_val:
            new_val = randint(0, 10)
        labels_with_noice[i] = new_val
        
    return labels_with_noice

In [34]:
make_noice_label(y_train, 0.8)

array([9, 0, 0, ..., 3, 0, 5], dtype=uint8)

In [35]:
y_train

array([9, 0, 0, ..., 3, 0, 5], dtype=uint8)